In [7]:
import random

from scipy.stats import norm

# Задача 1. Вариант 2.

Требуется аналитически решить задачу. Помимо этого нужно написать программу, которая будет считать данные вероятности приблизительно с помощью метода Монте-Карло. Особо любознательные могут задаться поиском (аналитическим) оптимального количества итераций, которых будет достаточно для получения ответа с достаточно «разумной» точностью. При решении задач могут быть полезны принципы произведения вероятностей и включений-исключений.

В городе живут n + 1 людей. Человек, условный «прародитель», пишет два письма случайно выбранным адресатам, которые образуют «первое» поколение. Те делают то же самое, в результате чего образуется «второе поколение». В общем, на каждое полученное письмо горожанин готовит два ответа и отправляет двум случайным жителям. Найти вероятность того, что прародитель не входит ни в одно из поколений с номерами 1, . . . , r.

## Аналитическое решение:
$$
\boxed{
P(A_M) = \left(\frac{n - 1}{n}\right)^{2^{r+1} - 4}
}
$$


In [8]:
def analytical_solution(n, r):
    """Возвращает вероятность того, что прародитель не входит ни в одно из поколений с номерами 1...r."""
    return ((n - 1) / n) ** (2 ** (r + 1) - 4)

In [14]:
def one_simulation(n, r):
    """Проводит одну симуляцию и возвращает True, если прародитель не входит ни в одно из поколений с номерами 1...r."""
    ancestor = 0
    senders = [ancestor]
    for t in range(1, r + 1):
        next_senders = []
        for s in senders:
            for _ in range(2):
                k = random.randrange(0, n)
                if k >= s:
                    recipient = k + 1
                else:
                    recipient = k
                next_senders.append(recipient)
        if ancestor in next_senders:
            return False
        senders = next_senders
    return True


In [22]:
def monte_carlo(n, r, trials):
    """Возвращает приближенную вероятность того, что прародитель не входит ни в одно из поколений с номерами 1...r методом Монте-Карло."""
    success = 0
    for _ in range(trials):
        if one_simulation(n, r):
            success += 1
    return success / trials


## Оценка числа итераций

Пусть $k$ - число успешных испытаний, $N$ - общее число испытаний, тогда оценка вероятности успеха:
$$
\hat{p} = \frac{k}{N}
$$

Математическое ожидание оценки:
$$
E[\hat{p}] = p
$$

Дисперсия оценки:
$$
Var(\hat{p}) = \frac{p(1 - p)}{N}
$$

Стандартное отклонение оценки:
$$
\sigma_{\hat{p}} = \sqrt{\frac{p(1 - p)}{N}}
$$


_Теорема Ляпунова 2.
Если случайная величина $X$ имеет конечные математическое ожидание $E[X]$ и дисперсию $D[X]$, то распределение среднего арифметического $\overline{X} = \frac{1}{n} \sum_{i=1}^n X_i$ вычисленного по наблюдавшимся значениям случайной величины в $n$ независимых испытаниях, проведённых в одинаковых условиях, при $n \to \infty$ приближается к нормальному с математическим ожиданием $E[X]$ и дисперсией $\frac{D[X]}{n}$._

По теореме Ляпунова, при большом числе испытаний, распределение оценки $\hat{p}$ приближается к нормальному распределению с параметрами $E[\hat{p}]$ и $Var(\hat{p})$.

Для определения необходимого числа итераций $N$ воспользуемся доверительным интервалом для оценки вероятности $p$. Мы хотим, чтобы с заданной доверительной вероятностью $1 - α$ наша оценка $\hat{p}$ отличалась от истинного значения p не более чем на заданную точность $ε$.

То есть:
$$
P(|\hat{p} - p|
\le \epsilon)
\ge 1 - \alpha
$$

Поскольку $\hat{p}$ имеет распределение, близкое к нормальному $N(p, p(1-p)/N)$, мы можем стандартизировать неравенство:
$$
P\left(\frac{|\hat{p} - p|}{\sqrt{p(1-p)/N}}
\le \frac{\epsilon}{\sqrt{p(1-p)/N}}\right)
\ge 1 - \alpha
$$

$$
Z
= \frac{\hat{p} - E[\hat{p}]}{\sqrt{Var(\hat{p})}}
= \frac{\hat{p} - p}{\sqrt{p(1-p)/N}}
$$
Таким образом, величина слева является модулем стандартной нормальной случайной величины $Z$. Для стандартного нормального распределения выполняется
$$
P(|Z| \le z_{\alpha/2}) = 1 - \alpha,
$$
где $z_{\alpha/2}$ — это квантиль уровня $1 - α/2$.

Отсюда получаем:
$$\frac{\epsilon}{\sqrt{p(1-p)/N}} \ge z_{\alpha/2}$$

Выразим $N$ из этого неравенства:
$$
\epsilon \sqrt{N}
\ge z_{\alpha/2} \sqrt{p(1-p)}
$$

Нам известно, что $p = \left(\frac{n - 1}{n}\right)^{2^{r+1} - 4}$. Подставляя это значение, получаем:
$$
N
\ge \frac{z_{\alpha/2}^2 p(1-p)}{\epsilon^2}
\ge \frac{
z_{\alpha/2}^2
\left(\frac{n - 1}{n}\right)
^{2^{r+1} - 4}
\left(1 - \left(\frac{n - 1}{n}\right)
^{2^{r+1} - 4}\right)}
{\epsilon^2}
$$

Таким образом, для достижения заданной точности $ε$ с доверительной вероятностью $1 - α$, необходимо провести $N$ испытаний, где $N$ определяется формулой:

$$
\boxed{
N =
\lceil{
\frac{
z_{\alpha/2}^2
\left(\frac{n - 1}{n}\right)
^{2^{r+1} - 4}
\left(1 - \left(\frac{n - 1}{n}\right)
^{2^{r+1} - 4}\right)}
{\epsilon^2}}\rceil}
$$



In [23]:
def N_required(n, r, epsilon, alpha):
    """Возвращает необходимое число испытаний для достижения заданной точности epsilon с доверительной вероятностью 1 - alpha."""
    p = ((n - 1) / n) ** (2 ** (r + 1) - 4)
    z_alpha_2 = norm.ppf(1 - alpha / 2)
    N = (z_alpha_2 ** 2 * p * (1 - p)) / (epsilon ** 2)
    return int(N) + 1

## Пример 1.
n = 100
r = 4
ε = 0.01
α = 0.001

In [92]:
n = 100
r = 4
eps = 0.01
alpha = 0.001
trials = N_required(n, r, eps, alpha)
print(f"Необходимое число испытаний для достижения точности {eps} с доверительной вероятностью {1 - alpha}: {trials}")
print(f"Аналитическое решение: P(A_M) = {((n - 1) / n) ** (2 ** (r + 1) - 4):.4f}")
print(f"Метод Монте-Карло: P(A_M) = {monte_carlo(n, r, trials):.4f} при {trials} испытаниях")

Необходимое число испытаний для достижения точности 0.01 с доверительной вероятностью 0.999: 20044
Аналитическое решение: P(A_M) = 0.7547
Метод Монте-Карло: P(A_M) = 0.7544 при 20044 испытаниях


## Пример 2.
n = 1000
r = 5
ε = 0.01
α = 0.001

In [91]:
n = 1000
r = 5
eps = 0.01
alpha = 0.001
trials = N_required(n, r, eps, alpha)
print(f"Необходимое число испытаний для достижения точности {eps} с доверительной вероятностью {1 - alpha}: {trials}")
print(f"Аналитическое решение: P(A_M) = {((n - 1) / n) ** (2 ** (r + 1) - 4):.4f}")
print(f"Метод Монте-Карло: P(A_M) = {monte_carlo(n, r, trials):.4f} при {trials} испытаниях")

Необходимое число испытаний для достижения точности 0.01 с доверительной вероятностью 0.999: 5941
Аналитическое решение: P(A_M) = 0.9417
Метод Монте-Карло: P(A_M) = 0.9438 при 5941 испытаниях


## Пример 2.
n = 10000
r = 15
ε = 0.01
α = 0.001

In [95]:
n = 10000
r = 12
eps = 0.01
alpha = 0.001
trials = N_required(n, r, eps, alpha)
print(f"Необходимое число испытаний для достижения точности {eps} с доверительной вероятностью {1 - alpha}: {trials}")
print(f"Аналитическое решение: P(A_M) = {((n - 1) / n) ** (2 ** (r + 1) - 4):.4f}")
print(f"Метод Монте-Карло: P(A_M) = {monte_carlo(n, r, trials):.4f} при {trials} испытаниях")

Необходимое число испытаний для достижения точности 0.01 с доверительной вероятностью 0.999: 26692
Аналитическое решение: P(A_M) = 0.4409
Метод Монте-Карло: P(A_M) = 0.4389 при 26692 испытаниях
